In [1]:
from pydantic import BaseModel

import _config
import sys
from pprint import pprint

class Row(BaseModel):
    ign: str  
    pfid: str
    mapfid: str
    go_rating: float


def as_playfab_id(player_id: int) -> str:
    return hex(player_id + 2**63).upper()[2:]


def as_player_id(playfab_id: str) -> int:
            player_id = int(playfab_id,16) #playfabs are 16 digit Hex numbers (64 bit int)
            player_id -= 2**63  # db is signed so make this range from -2^63 to + 2^63
            return player_id

In [2]:
from sqlalchemy import create_engine
from sqlmodel import SQLModel
import importlib
importlib.reload(_config)

engine = create_engine(_config.godb_url, echo=_config.godb_echo)
SQLModel.metadata.create_all(engine)



In [5]:
import csv
from sqlmodel import Session
from go.bot.models import GoRatings

file_path = '../data/after_s10/go_s10_ratings.csv'

with open(file_path, mode='r') as file:

    csv_reader = csv.DictReader(f=file, fieldnames='ign pfid mapfid go_rating rating2'.split())
    
    with Session(engine) as session:
    
        for i,row in enumerate(csv_reader):
            if i==0:
                continue
            print(row)
            row = Row(**row)
            print(i, row)
            pf_player_id = as_player_id(row.pfid)
            rating = GoRatings(pf_player_id=pf_player_id, season="GOP1 S10", rating_type="combined", go_rating=row.go_rating)
            session.add(rating)
            if i%100 == 0:
                session.commit()
        session.commit()


{'ign': 'TTP.Guramichi15', 'pfid': '79ED45AD17C8483B', 'mapfid': '79ED45AD17C8483B', 'go_rating': '1946', 'rating2': '2699'}
1 ign='TTP.Guramichi15' pfid='79ED45AD17C8483B' mapfid='79ED45AD17C8483B' go_rating=1946.0
{'ign': 'MyVRUser12', 'pfid': '764DD262873FF5FD', 'mapfid': '764DD262873FF5FD', 'go_rating': '1920', 'rating2': '2690'}
2 ign='MyVRUser12' pfid='764DD262873FF5FD' mapfid='764DD262873FF5FD' go_rating=1920.0
{'ign': 'Gabe_Watts.NB', 'pfid': '897EC5B9C24E425E', 'mapfid': '897EC5B9C24E425E', 'go_rating': '1883', 'rating2': '2613'}
3 ign='Gabe_Watts.NB' pfid='897EC5B9C24E425E' mapfid='897EC5B9C24E425E' go_rating=1883.0
{'ign': 'Scanter', 'pfid': '9CC35B4B3A8E41D1', 'mapfid': '9CC35B4B3A8E41D1', 'go_rating': '1880', 'rating2': '2768'}
4 ign='Scanter' pfid='9CC35B4B3A8E41D1' mapfid='9CC35B4B3A8E41D1' go_rating=1880.0
{'ign': 'Legends_1', 'pfid': '8F5542CC5FA8A54F', 'mapfid': '8F5542CC5FA8A54F', 'go_rating': '1849', 'rating2': '2638'}
5 ign='Legends_1' pfid='8F5542CC5FA8A54F' mapfi